# Reducing pointing data

During the SUMMIT-4829 run, we obtained data at lower elevation to improve the existing pointing model. 

The data was taken in a slightly different way. Instead of centering the start in the CCD and storing the data for the pointing model we simply ran the wavefront analysis to collimate the optics, registered the position in the pointing and took an acquisition image, so we could later measure the offset and apply that to the registered position.

This notebooks is intended to gather the information about the data taken during the run. Basically we need to all the registered positions in associated acquisition images.

The next step will be to measure the position of the brightest start in the field, compute the offset with respect to the center of the field and add that to the data generated by the pointing. This will be done on a separate notebook.

## Parameterized notebook

This notebook is parameterized, which means it could be run with tools like Papermill as part of a data analysis pipeline.

## Requirements

In order to run this notebook you will need an updated version of `QuickFrameMeasurementTask`. 
By the time of this writting this library was still not integrated to the DM stack, so you may need to set it up manually. 

Assuming you are in one of the nublado environments (tts, nts, summit, etc), open a terminal and do the following:

```console
source ${LOADSTACK}
cd ${HOME}/WORK
git clone https://github.com/lsst-sitcom/rapid_analysis.git
cd rapid_analysis/
eups declare -r . -t $USER
```

Then, on your `${HOME}/notebooks/.user_setups` file, add the following like:

```
setup rapid_analysis -t $USER
```

In [ ]:
import os 

import numpy as np
import pandas as pd

from astropy.time import Time
from astropy import units as u
from datetime import timedelta, datetime

import lsst_efd_client

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from pandas.plotting import register_matplotlib_converters

from lsst.geom import PointD

from lsst.pipe.tasks.quickFrameMeasurement import QuickFrameMeasurementTask

from lsst.ts.observing.utilities.auxtel.latiss.getters import get_image
from lsst.ts.observing.utilities.auxtel.latiss.utils import (
    calculate_xy_offsets,
    parse_obs_id,
)
from lsst.ts.observatory.control.constants.latiss_constants import boresight #, pixel_scale



In [ ]:
%matplotlib inline

In [ ]:
client = lsst_efd_client.EfdClient('ldf_stable_efd')

## Notebook Parameters

The next cells define the notebook parameters

## Date of the observation

In [ ]:
year=2021
month=3
day=22

### Pointing file name

In [ ]:
pointing_file = f"data/20210323/AT_point_file_20230323.dat"

### Define timestamp when the data was taken

The next cell defines the dates when the data was taken, it is used by tthe following query to determine when to look for pointing component data registration

### SUMMIT-4829

```
start = Time('2021-02-18T00:00:00')
end = Time('2021-02-20T00:00:00')
```

### SUMMIT-5025

```
start = Time('2021-03-22T00:00:00')
end = Time('2021-03-25T00:00:00')
```

In [ ]:
start = Time(f"{year}-{month:02d}-{day:02d}T00:00:00")
end = Time(f"{year}-{month:02d}-{day+3:02d}T00:00:00")

In [ ]:
print(start, end)

# Data analysis

Look from when pointAddData command was sent to the pointing. These will mark the times when we registered the positions.

In [ ]:
timestamp = f"time >= '{start}+00:00' AND time <= '{end}+00:00'"
query = f'SELECT "private_host" FROM "efd"."autogen"."lsst.sal.ATPtg.command_pointAddData" WHERE {timestamp}'
query_ack = f'SELECT "cmdtype", "ack" FROM "efd"."autogen"."lsst.sal.ATPtg.ackcmd" WHERE {timestamp} AND cmdtype = 24 AND ack != 300'

In [ ]:
print(query)
print(query_ack)

In [ ]:
point_add_data = await client.influx_client.query(query)

In [ ]:
point_add_data_ack = await client.influx_client.query(query_ack)

In [ ]:
len(point_add_data_ack), len(point_add_data)

In [ ]:
command_ok = []
for i in range(len(point_add_data)):
    if point_add_data_ack.ack[i] == 303:
        print(f"Command OK: {point_add_data.private_host[i]}")
        command_ok.append(i)
    else:
        print(f"Command FAILED: {point_add_data.private_host[i]}")

In [ ]:
point_add_data_ok = point_add_data.loc[point_add_data.index[command_ok]]

In [ ]:
print(len(point_add_data_ok))

## Finding acquisition images

Now we have the timestamps for when the telescope position was registered, we need to find the acquisition images.

The images where taken before registering the position so we need to look ~40s before the command was sent.

In [ ]:
acq_obsid = []
elevation = []
rotator_1 = []
rotator_2 = []
for time_reg in point_add_data_ok.index:
    obsid = await client.select_time_series('lsst.sal.ATArchiver.logevent_imageInOODS', ["obsid"], 
                                     Time(time_reg).tai - timedelta(seconds=40), Time(time_reg).tai + timedelta(seconds=40))
    
    # It may happen that a image is not taken (or fails to be taken) when we register the position.
    # In this cases, add `None` to the acq_obsid list. These data will later be ignored.
    if hasattr(obsid, "obsid"):
        acq_obsid.append(obsid.obsid[-1])
    else:
        acq_obsid.append(None)

    mount_Nasmyth_Encoders = await client.select_packed_time_series("lsst.sal.ATMCS.mount_Nasmyth_Encoders",
                                                                    ["nasmyth1CalculatedAngle", 
                                                                     "nasmyth2CalculatedAngle"],
                                                                    Time(time_reg).tai - timedelta(seconds=40), 
                                                                    Time(time_reg).tai)
    rotator_1.append(np.mean(mount_Nasmyth_Encoders["nasmyth1CalculatedAngle"]))
    rotator_2.append(np.mean(mount_Nasmyth_Encoders["nasmyth2CalculatedAngle"]))
    elevationCalculatedAngle = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders",
                                                                      ["elevationCalculatedAngle"],
                                                                      Time(time_reg).tai - timedelta(seconds=40), 
                                                                      Time(time_reg).tai
                                                                     )
    elevation.append(np.mean(elevationCalculatedAngle["elevationCalculatedAngle"]))


In [ ]:
good_data = np.where(np.array([val is not None for val in acq_obsid]))[0]

In [ ]:
len(good_data)

## Measuring star position on each image

In [ ]:
def rotation_matrix(angle):
    """Rotation matrix.
    """
    return np.array(
        [
            [np.cos(np.radians(angle)), -np.sin(np.radians(angle)), 0.0],
            [np.sin(np.radians(angle)), np.cos(np.radians(angle)), 0.0],
            [0.0, 0.0, 1.0],
        ]
    )

In [ ]:
qm_config = QuickFrameMeasurementTask.ConfigClass()

In [ ]:
qm = QuickFrameMeasurementTask(config=qm_config)

In [ ]:
brightest_source_centroid = []
exposures = []

for obs_id in acq_obsid:
    if obs_id is None:
        continue
    day_obs, seq_num = parse_obs_id(obs_id)[-2:]
    exp = await get_image(
            dict(dayObs=day_obs, seqNum=seq_num),
            datapath="/project/shared/auxTel/",
            timeout=10,
            runBestEffortIsr=True,
        )
    result = qm.run(exp)
    exposures.append(exp)
    brightest_source_centroid.append(result)

In [ ]:
print("Done")

## Create a plot with the images

The next cell will create a plot with the images. The center (boresight) is marked with a "+" sign and the position of the brightest source with an open red circly.

In [ ]:
n_y_panels = (np.ceil(np.sqrt(len(exposures)))+1)

In [ ]:
n_x_panels = len(exposures) / n_y_panels

In [ ]:
n_x_panels, n_y_panels = int(n_x_panels), int(n_y_panels)

In [ ]:
print(f"Contain {len(exposures)} exposures. Will be displayed in a {n_x_panels}x{n_y_panels} grid.")

In [ ]:
fig = plt.figure(figsize=(14, 10))

for idx in range(len(exposures)):
    axis = plt.subplot(n_x_panels,n_y_panels,idx+1)
    plt.imshow(exposures[idx].getImage().array, origin='lower',cmap='gray', norm=LogNorm())
    plt.scatter(*brightest_source_centroid[idx].brightestObjCentroid, facecolors='none', s=80,edgecolors='r')
    plt.scatter(boresight.getX(), boresight.getY(), marker="+")
    axis.set_axis_off()
    axis.set_title(f"[{idx+1}]:{elevation[good_data[idx]]:0.1f}")


## Compute azel offset

Each entry in `brightest_source_centroid` contains the centroid of thn_x_panelsst source in the field in pixel coordinates.

We need to compute the distance to the center of the field and then convert that from xy into azel. It is this azel offset that we need to apply to the pointing table.


In [ ]:
angle = np.array(elevation) - np.array(rotator_2) + 90.0

The next cell will select only the angles for those data points for which we got images above.

In [ ]:
angle = angle[good_data]

In [ ]:
azel_correction = np.zeros((2,len(brightest_source_centroid)))

for i, source_xy in enumerate(brightest_source_centroid):
    dx_arcsec, dy_arcsec = calculate_xy_offsets(PointD(*source_xy.brightestObjCentroid), boresight)

    # We are using rotator 2 so we must apply a negative sign on the x-axis offset.
    # The equation bellow return offset in elevation/azimuth.
    elaz_offset = np.matmul((-dx_arcsec, dy_arcsec), rotation_matrix(angle[i])[:2,:2])*u.arcsec
    # We want to store the offset in azel format, so we reverse the result given above.
    azel_correction[0][i] = elaz_offset[1].to(u.deg).value
    azel_correction[1][i] = elaz_offset[0].to(u.deg).value

# The following was verified with the pointing component. When we add an offset of X arcsec in azimuth it 
# results in a negative offset in the axis. When we make a positive offset in elevation is results in a 
# positive offset in the axis. The pointing takes care of the cos(elevation) dependency when we apply the
# offset, but we need to take care of it here since we want to apply a correction to the axis directly.
azel_correction[0] *= -1./np.cos(np.radians(np.array(elevation)[good_data]))
azel_correction[1] *= 1.

## Apply correction to pointing data

Now that the offsets are computed in az/el, we need to read the data from the pointing table and apply the offset to the appropriate columns. 

Skip the first `skiprows=4` lines of the file, which contain header information.

Read `max_rows=len(point_add_data_ok)`. This avoids the issue of the last line in the pointing file being `END`.

In the end get only those data for which there is a paired image.

In [ ]:
pointing_file_data = np.loadtxt(
    pointing_file,
    skiprows=4,
    max_rows=len(point_add_data_ok),
    unpack=False)[good_data].T

In [ ]:
delta_az = pointing_file_data[0]-pointing_file_data[2]
delta_el = pointing_file_data[1]-pointing_file_data[3]

In [ ]:
for az, el, d_az, d_el in zip(delta_az, delta_el, azel_correction[0], azel_correction[1]):
    plt.arrow(az, el, -d_az, -d_el)

The data that needs correcting are the 3rd and 4th columns, which contains the correct az and el of the telescope. 

In [ ]:
pointing_file_data = np.loadtxt(
    pointing_file,
    skiprows=4,
    max_rows=len(point_add_data_ok),
    unpack=False)[good_data].T

pointing_file_data[2] += azel_correction[0]
pointing_file_data[3] += azel_correction[1]

In [ ]:
out_pointing_file, ext = os.path.splitext(pointing_file)

In [ ]:
print(out_pointing_file, ext)

We need to add the header and tail of the pointing file.

In [ ]:
header = f"""LSST Auxiliary Telescope, {year} {month} {day} UTC 1 41 1
: ALTAZ
: ROTNL
-30 14 40.3
"""
tail = "END"
with open(f"{out_pointing_file}_rev{ext}", "w") as fp:
    fp.write(header)
    np.savetxt(fp, pointing_file_data.T, fmt="%011.7f")
    fp.write(tail)

## End

The file is now ready to be analysed with tpoint to produce a new pointing model.